In [1]:
import pandas as pd
import numpy as np
from IPython.display import display

#Plots:
import plotly.graph_objects as go
import plotly.express as px



In [2]:
# pd.set_option("display.max_rows", 999)
# pd.set_option('max_colwidth',100)

## Importing pre-processed csv file

In [3]:
# taking csv prepared earlier to explore data 

filepath = "Enron_Kaggle_dataset/email_features.csv"
# Read the data into a pandas dataframe called emails

df_email = pd.read_csv(filepath)

headers = [header for header in df_email.columns]

print("Successfully loaded {} rows and {} columns!".format(df_email.shape[0], df_email.shape[1]))

print(display(df_email.head(3)))

#Convert date column to datetime 

df_email["Date"] = pd.to_datetime(df_email["Date"])

Successfully loaded 517401 rows and 12 columns!


,file,message,Message-Body,X-From,employee,Date,Subject,X-Folder,X-To,X-cc,X-bc,X-Origin
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,here is our forecast\n\n,Phillip K Allen,allen-p,2001-05-14 23:39:00+00:00,NaN,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Tim Belden <Tim Belden/Enron@EnronXGate>,NaN,NaN,Allen-P
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,traveling to have a business meeting takes the...,Phillip K Allen,allen-p,2001-05-04 20:51:00+00:00,Re:,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",John J Lavorato <John J Lavorato/ENRON@enronXg...,NaN,NaN,Allen-P
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,test successful. way to go!!!,Phillip K Allen,allen-p,2000-10-18 10:00:00+00:00,Re: test,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Leah Van Arsdall,NaN,NaN,Allen-P


None


In [4]:
### Findind NAN values: 

df_email.isna().sum()

file                 0
message              0
Message-Body         0
X-From              29
employee             0
Date                 0
Subject          19187
X-Folder            29
X-To              9153
X-cc            388515
X-bc            517401
X-Origin            29
dtype: int64

In [5]:
### Function to clean machine code from email's content:

def cleanemail(email):
    return ' '.join([s.strip() for s in email.splitlines()][15:])

df_email['email-body'] = df_email['message'].apply(cleanemail)

In [6]:
### Cleaning csv by droping non-relevant columns:

df_email = df_email.drop(columns=['message', 'file', 'X-bc'])
df_email = df_email.drop(columns=['Message-Body'])


In [7]:
### Verifying NAN values :

df_email.isna().sum()

X-From            29
employee           0
Date               0
Subject        19187
X-Folder          29
X-To            9153
X-cc          388515
X-Origin          29
email-body         0
dtype: int64

In [8]:
### Count of available email's subject: 

df_email.Subject.value_counts()

RE:                                                                 6477
Re:                                                                 6306
Demand Ken Lay Donate Proceeds from Enron Stock Sales               1124
FW:                                                                  938
Schedule Crawler: HourAhead Failure                                  900
                                                                    ... 
Ship any order for only $4.99                                          1
Mtg: Max Yzaguirre will call you                                       1
FW: IMPORTANT - R: Drive Utilization                                   1
Subscription probe for list AllActiveMA (larry.may@enron.com:19)       1
Dionne L Williams/TMG/CSC is out of the office.                        1
Name: Subject, Length: 159289, dtype: int64

## Top email senders

In [9]:
df_email['X-From'].value_counts().head(5)

Kay Mann               16324
Vince J Kaminski       14367
Jeff Dasovich           9701
Enron Announcements     8535
Tana Jones              8105
Name: X-From, dtype: int64

## Top email recipients

In [10]:
df_email['X-To'].value_counts().head(10)

pete.davis@enron.com                                     5334
vkaminski@aol.com                                        4125
Tana Jones                                               4099
Davis, Pete </O=ENRON/OU=NA/CN=RECIPIENTS/CN=PDAVIS1>    3806
All Enron Worldwide                                      3243
Kate Symes                                               3158
Sara Shackleton                                          2936
Steven J Kean                                            2892
Mark Taylor                                              2485
Jeff Dasovich                                            2451
Name: X-To, dtype: int64

## Relationship between senders and recipients:

In [11]:
PKA = df_email.groupby(['X-From', 'X-To'])
PKA.get_group(('Kay Mann', 'Sara Shackleton')).head(3)

,X-From,employee,Date,Subject,X-Folder,X-To,X-cc,X-Origin,email-body
302120,Kay Mann,mann-k,2000-09-19 09:12:00+00:00,ISDA for Peoples/Enron activities,\Kay_Mann_June2001_4\Notes Folders\'sent mail,Sara Shackleton,NaN,MANN-K,"Hi Sara, I believe we still have an interest..."
302317,Kay Mann,mann-k,2000-08-30 08:49:00+00:00,"Re: Midwest Energy Hub LLC (""Midwest"")",\Kay_Mann_June2001_4\Notes Folders\'sent mail,Sara Shackleton,NaN,MANN-K,Feels funny emailing someone so close. I'm a...
302319,Kay Mann,mann-k,2000-08-29 19:09:00+00:00,"Re: Midwest Energy Hub LLC (""Midwest"")",\Kay_Mann_June2001_4\Notes Folders\'sent mail,Sara Shackleton,Gregg Penman,MANN-K,"X-Origin: MANN-K X-FileName: kmann.nsf Sara, ..."


## Reading a random email

In [12]:
df_email['email-body'][28]

' Reagan,  Just wanted to give you an update.  I have changed the unit mix to include some 1 bedrooms and reduced the number of buildings to 12.  Kipp Flores is working on the construction drawings.  At the same time I am pursuing FHA financing.  Once the construction drawings are complete I will send them to you for a revised bid.  Your original bid was competitive and I am still attracted to your firm because of your strong local presence and contacts.  Phillip'

## Saving subset of cleaner data as a csv file:

In [13]:
### Uncomment if neccesary:

# subset = df_email.loc[np.random.choice(df_email.index, 10000, replace=False)] # 1000 emails
# subset.to_csv('enron_subset_10kemails.csv',index=False)

## Company statistics

In [14]:
### Emails sent by every employee: 

emails_sent = df_email.groupby('employee')[['email-body']].nunique()



In [15]:
### Total of employes in the company: 

df_email['employee'].nunique()

150

In [16]:
### Unique email-body messages: 

print(df_email['email-body'].nunique())

print('Duplicated emails:', df_email['email-body'].shape[0] - df_email['email-body'].nunique() )

313217
Duplicated emails: 204184


In [17]:
### Dropping duplicates in email-body:

df_email = df_email.drop_duplicates(['email-body'])

In [18]:
type(df_email['Date'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [19]:
### Adding classification by year to dataframe:

df_email['year'] = pd.DatetimeIndex(df_email['Date']).year


<ipython-input-19-3a62514d9bae>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_email['year'] = pd.DatetimeIndex(df_email['Date']).year


In [20]:
# #### Functions to plot relevant data:

# def time_workload(dataset, columns, time_frame):
    
# #     time_frame = 'year'
# #     dataset = 'df_email'
# #     columns = 'email-body'
# #     year you want to explore



#     ax = f"{dataset}.groupby({dataset}['Date'].dt.{time_frame}){[[columns]]}.count()"
#     ax = eval(ax)

    
#     return ax


# def bar_plot_interactive(dataset, y, test):
    
#     fig = px.bar(dataset, y= y, title= test)
#     fig.update_xaxes(type='category')
    
#     return fig.show()

In [267]:
#### Functions to plot relevant data:

def time_workload(dataset, columns, time_frame, year_of_interest, normalized = True):
    
#     time_frame = 'year'
#     dataset = 'df_email'
#     columns = 'email-body'
#     year you want to explore

    if year_of_interest != None:
        
        ### Data frame with information for year of interest:
        order = f"{dataset}.loc[{dataset}['year'] == {year_of_interest}]"
        data_year = eval(order)
        
        ### Data frame with total for year of interest:
        order2 = f"data_year.groupby(data_year['Date'].dt.{time_frame}){[[columns]]}.count()"
        data = eval(order2)

    else:
        
        data_historical = f"{dataset}.groupby({dataset}['Date'].dt.{time_frame}){[[columns]]}.count()"
        data = eval(data_historical)
    
    if normalized == True:
        
        ### Data normalization (To display percentage)
        total = data.sum()
        data = data/total

    return data

def bar_plot_interactive(dataset, y, title, format=None):
    
    fig = px.bar(dataset, y= y, title= title)
    fig.update_xaxes(type='category')
    
    if format == "%":
        fig.update_layout(yaxis_tickformat = '%')

    return fig

In [285]:
#### Example of yearly plot: 

data = time_workload('df_email', 'email-body', 'year', None, normalized=True) ## When year_of_interest is None historical data is showed

fig = bar_plot_interactive(data, 'email-body', 'Emails sent by year')


### Layout for figure:

fig.update_layout(
    title="Plot Title",
    xaxis_title="X Axis Title",
    yaxis_title="X Axis Title",
    legend_title="Legend Title")

### If more details are needed:

# fig.update_layout(
#     title="Plot Title",
#     xaxis_title="X Axis Title",
#     yaxis_title="X Axis Title",
#     legend_title="Legend Title",
#     font=dict(
#         family="Arial",
#         size=18,
#         color="LightSlateGrey"
#     )
# )

fig.show()

## Emails on a timeframe: 

In [286]:
### Updating figure to timeframe of interest :

data = data[data.index > 1998]
data = data[data.index < 2003]

fig = bar_plot_interactive(data,'email-body', 'Emails sent by year', '%')

### Layout for figure:

fig.update_layout(
    xaxis_title="Year",
    yaxis_title="Emails",
    legend_title="Legend Title")

fig.show()

In [288]:
#### Monthly: 

year_of_interest = 2001
data = time_workload('df_email', 'email-body', 'month', year_of_interest)
fig = bar_plot_interactive(data, 'email-body', 'Emails  sent by month', '%')

### Layout for figure:

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Emails",
    legend_title="Legend Title")

fig.show()

In [293]:
#### Daily: (ooking at the distribution in all the year)

year_of_interest = 2001

data = time_workload('df_email', 'email-body', 'day', year_of_interest)
fig = bar_plot_interactive(data, 'email-body', 'Emails  sent by day', '%')

### Layout for figure:

fig.update_layout(
    xaxis_title="Day of the month",
    yaxis_title="Emails",
    legend_title="Legend Title")

fig.show()

###Maybe not helpful in this way

In [294]:
#### Hourly:

year_of_interest = 2000
data = time_workload('df_email', 'email-body', 'hour', year_of_interest)
fig = bar_plot_interactive(data, 'email-body', 'Emails sent by hour', '%')

### Layout for figure:

fig.update_layout(
    xaxis_title="Hour of the day",
    yaxis_title="Emails",
    legend_title="Legend Title")

fig.show()



In [301]:
#### Weekly:

year_of_interest = 2001
data = time_workload('df_email', 'email-body', isocalendar().'week', year_of_interest)
bar_plot_interactive(data, 'email-body', 'Emails sent per week of the year', '%')

### Layout for figure:

fig.update_layout(
    xaxis_title="Week of the year",
    yaxis_title="Emails",
    legend_title="Legend Title")

fig.show()

SyntaxError: invalid syntax (<ipython-input-301-79f032033597>, line 4)